# Air Quality Data Analysis Workshop
## Predicting PM2.5 Levels in African Cities

**Learning Objectives:**
By completing this workshop, you will be able to:
- Handle time-series data with geographical groupings
- Understand the challenges of missing data in time-series
- Apply appropriate cross-validation strategies for grouped data
- Perform feature engineering on temporal and geographical data
- Build and evaluate regression models for environmental data
 
**Context:**
Air quality is a critical environmental and health issue. PM2.5 (particulate matter ≤ 2.5 micrometers) is particularly dangerous as these tiny particles can penetrate deep into lungs and bloodstream.

This dataset contains air quality measurements from 4 African cities over time. Unlike typical machine learning problems, this data has **temporal dependencies** and **geographical groupings** 
that require special handling.

***
# 1. Data Loading and Initial Exploration
 
Let's start by importing the necessary libraries and loading our data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Configuration class to keep our paths and column names organized
class Config:
    path = "../data/"
    id_col = "id"
    target_col = "pm2_5"

In [ ]:
# TODO Load the training and test datasets
# The training set contains our target variable (pm2_5) that we want to predict
# The test set is what we'll make predictions on (without knowing the true pm2_5 values)

train = pd.read_csv(Config.path + "train.csv")
test = pd.read_csv(Config.path + "test.csv")

**Exercise:** Explore the basic structure of our data.

Use pandas methods to understand:
- What columns do we have?
- What are the data types?
- How much data is in each dataset?

In [ ]:
# TODO Print the shape of these datasets

print(f"Training data shape: {train.shape}")
print(f"Test data shape: {test.shape}")

In [ ]:
# TODO Display basic information about the training dataset

train.info()

In [ ]:
# TODO Display the first few rows to understand the data structure
train.head()

**Exercise:** Explore the geographical and temporal scope.

Find out:
- Which countries and cities are in our dataset?
- What is the time range of our data?


In [ ]:
# TODO Print unique countries and cities
print("Unique countries:", train['country'].unique())
print("Unique cities:", train['city'].unique())

In [ ]:
# TODO Print the time range of our data
print(f"Date range: {train['date'].min()} to {train['date'].max()}")


***
# 2. Data Visualization and Pattern Discovery
  
Before building models, we need to understand our data patterns. For air quality data, it's crucial to understand temporal trends and city-specific patterns. However, creating effective data visualizations requires more than just plotting data points—it demands thoughtful design choices that follow established principles for clear communication.

**Exercise:** Configure global visualization styling for professional charts

Effective data visualization requires consistent styling that eliminates visual clutter and ensures accessibility. As demonstrated in the air quality workshop, establishing global styling parameters creates a foundation for clean, professional charts throughout your analysis. This configuration applies principles of visual hierarchy and accessibility without requiring manual styling for each individual plot.

**Documentation references:**
- [Seaborn style control](https://seaborn.pydata.org/generated/seaborn.set_style.html)
- [Seaborn color palettes](https://seaborn.pydata.org/generated/seaborn.set_palette.html)
- [Colorblind-friendly design](https://seaborn.pydata.org/tutorial/color_palettes.html#qualitative-color-palettes)

Configure global styling settings by implementing a clean "white" background style that removes unnecessary visual elements while maintaining essential chart components. Remove grid lines and top/right spines to reduce clutter, but preserve left and bottom spines for data reference. Set slightly thicker axis lines for subtle emphasis. Additionally, apply a colorblind-friendly palette to ensure accessibility for users with color vision deficiency, making your visualizations inclusive for approximately 8% of the population.

In [ ]:
# TODO Set global seaborn style - clean background without grid
sns.set_style("white", {
    "axes.grid": False,           # Remove grid lines
    "axes.spines.left": True,     # Keep left spine
    "axes.spines.bottom": True,   # Keep bottom spine
    "axes.spines.top": False,     # Remove top spine
    "axes.spines.right": False,   # Remove right spine
    "axes.linewidth": 1.2,        # Slightly thicker axis lines
})

In [ ]:
# TODO Set inclusive and colorblind-friendly palette
# Using 'colorblind' palette which is accessible to colorblind users
sns.set_palette("colorblind")

**Exercise:** Select appropriate visualization for air quality data analysis

Air quality data contains PM2.5 measurements taken at discrete time points, requiring careful visualization choices to avoid misrepresentation. Understanding which chart type best represents your data prevents misleading interpretations about pollution patterns. Consider how air quality monitoring works: measurements are individual observations at specific moments, not continuous streams of data.

**Documentation references:**
- [Seaborn plot types overview](https://seaborn.pydata.org/tutorial/function_overview.html)
- [Matplotlib plot gallery](https://matplotlib.org/stable/gallery/index.html)
- [Choosing the right chart type](https://seaborn.pydata.org/tutorial/introduction.html#choosing-plot-types)

Examine your air quality dataset and identify the most appropriate visualization for analyzing PM2.5 levels over time for a single city. Consider these options:
- **Line plot**: Connects all measurement points with continuous lines
- **Scatter plot**: Shows individual measurements as discrete points without connection
- **Bar plot**: Shows each measurement as a separate categorical value
- **Histogram**: Shows distribution of PM2.5 values across time periods

Which visualization best represents the discrete nature of air quality measurements while still showing temporal patterns? Consider that PM2.5 readings are individual observations at specific times, not continuous processes, yet you want to identify underlying trends without implying false continuity between measurements.

**Exercise:** Implement professional air quality visualization

Implement your selected visualization technique to display PM2.5 levels over time for Lagos city, ensuring the representation accurately reflects the measurement methodology. Your visualization should clearly show individual observations while maintaining clean, accessible design principles and scientific accuracy.

**Documentation references:**
- [plt.figure()](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.figure.html) - Create figure and axes for visualization
- [Matplotlib plotting functions](https://matplotlib.org/stable/api/pyplot_summary.html) - Various plotting methods available
- [Seaborn statistical plotting](https://seaborn.pydata.org/api.html) - High-level statistical visualization functions
- [Adding trend analysis](https://numpy.org/doc/stable/reference/routines.polynomials.polynomial.html) - Mathematical functions for pattern identification

In [ ]:
# TODO Write a function to plot air quality data for a specific city

def plot_air_quality_subplot(train_data, city_name, ax):
    """
    Create air quality visualization for use in subplots

    Parameters:
    - train_data: DataFrame containing training data with 'date', 'pm2_5', 'city' columns
    - city_name: Name of the city to filter data for
    - ax: Matplotlib Axes object to plot on

    Returns:
    - ax: The Axes object with the plot for further customization
    
    ADAPTATIONS FOR SUBPLOT:
    - Receives ax parameter instead of creating plt.figure()
    - Uses ax.scatter() and ax.plot() instead of plt.scatter() and plt.plot()
    - Returns the ax object for further customization
    - Removes plt.show() (handled by parent function)
    """
    
    # Step 1: Prepare data for the specified city
    df_city = train_data[train_data['city'] == city_name].copy()
    df_city = df_city.sort_values('date')
    df_city['date_numeric'] = pd.to_datetime(df_city['date']).map(pd.Timestamp.toordinal)
    
    # Step 2: Create a plot for discrete measurements
    ax.scatter(df_city['date'], df_city['pm2_5'], 
               alpha=0.6, 
               s=15,  # Smaller points for subplot
               color=sns.color_palette("colorblind")[0],
               edgecolors='white',
               linewidth=0.5,
               label='Daily Measurements')
    
    # Step 3: Add polynomial trend line to show underlying pattern
    z = np.polyfit(df_city['date_numeric'], df_city['pm2_5'], 2)
    p = np.poly1d(z)
    ax.plot(df_city['date'], p(df_city['date_numeric']), 
            color=sns.color_palette("colorblind")[1],  
            linewidth=1,
            alpha=0.9,
            linestyle='--',  # Dashed to distinguish from scatter
            label='Trend Line')
    
    # Step 4: Add clear, descriptive title
    ax.set_title(f"PM2.5 Evolution in {city_name}", 
                 fontsize=12, fontweight='bold')
    
    # Step 5: Label axes with units
    ax.set_xlabel("Date", fontsize=10)
    ax.set_ylabel("PM2.5 (µg/m³)", fontsize=10)
    
    # Step 6: Improve date readability
    ax.tick_params(axis='x', rotation=0)
    ax.xaxis.set_major_locator(plt.MaxNLocator(4))
    
    # Step 7: Add legend (optional for subplots)
    ax.legend(frameon=False, loc='upper right', fontsize=8)
    
    return ax

In [ ]:
# TODO Create a single subplot for Lagos
fig, ax = plt.subplots(1, 1, figsize=(15, 8))
plot_air_quality_subplot(train, "Lagos", ax)
plt.show()

**Exercise:** Create multi-city air quality comparison visualization

Comparing air quality across multiple cities requires careful visual design to avoid overwhelming the audience. Using separate subplots for each city creates visual clutter and makes direct comparison difficult.



In [ ]:
# TODO Get all unique cities from the dataset
cities = train['city'].unique()

To produce the figure :
- Create 2x2 subplot structure for all cities
- Plot each city using the subplot function
- Add overall title for the entire figure
- Optimize layout to prevent overlap and show the plot

In [ ]:
# TODO Create the whole figure
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for i, city in enumerate(cities):
    plot_air_quality_subplot(train, city, axes[i])

fig.suptitle("PM2.5 Air Quality with Trends Across African Cities", 
             fontsize=16, fontweight='bold', y=0.98)

plt.tight_layout()
plt.subplots_adjust(top=0.93)  # Make room for suptitle
plt.show()

**Exercise:** Improve subplot comparability

Look at the current multi-city visualization. What problem do you observe when trying to compare PM2.5 levels between different cities in the current visualization?

To make visual comparison possible, all subplots need the same y-axis limits.

Modify the code above to:
- Calculate the global minimum and maximum PM2.5 values across all cities before creating the subplots
- Apply these fixed limits with `set_ylim` to each subplot's y-axis to ensure consistent scaling

In [ ]:
# TODO Create 2x2 subplot structure for all cities with the same max y value
y_max = train['pm2_5'].max()

cities = train['city'].unique()

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel() 


for i, city in enumerate(cities):
    axes[i].set_ylim(0, y_max)
    plot_air_quality_subplot(train, city, axes[i])

fig.suptitle("PM2.5 Air Quality with Trends Across African Cities", 
             fontsize=16, fontweight='bold', y=0.98)

plt.tight_layout()
plt.subplots_adjust(top=0.93)
plt.show()

***
# 3. Data Cleaning and Missing Value Handling
 
Real-world data often contains missing values that can significantly impact model performance. For time-series air quality data, missing values are particularly challenging because they can break temporal patterns and relationships. We need specialized approaches that preserve the sequential nature of our data while maintaining geographic consistency across different monitoring stations.

## 3.1 Missing Data Exploration

Understanding missing data patterns is crucial before applying any cleaning strategy. Identifying which variables have missing data and their patterns helps us choose appropriate imputation strategies.

**Documentation references:**
- [pandas.DataFrame.isnull()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isnull.html)
- [Missing data handling in pandas](https://pandas.pydata.org/docs/user_guide/missing_data.html)

**Exercise:** Identify missing data patterns across variables
- Which columns have missing values?
- What percentage of data is missing in each column?
  
Generate a two-column DataFrame summarizing missing data patterns in the training dataset  with `Missing_Count` (absolute number of missing values) and `Missing_Percentage` (percentage of missing data per column). Sort the results by missing percentage in descending order and display only columns that contain at least one missing value to identify data quality issues.


In [ ]:
# TODO Generate a summary of missing data in the training set
 
missing_data = train.isnull().sum()
missing_percentage = (missing_data / len(train) * 100).round(2)

In [ ]:
# Create a summary DataFrame for missing data

missing_summary = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percentage': missing_percentage
}).sort_values('Missing_Percentage', ascending=False)

print("Missing data summary:")
print(missing_summary[missing_summary['Missing_Count'] > 0].to_string())

**Exercise:** Remove columns with excessive missing data
 
Variables with too much missing data provide little useful information. Removing them prevents our models from learning unreliable patterns based on sparse data.
Remove columns with more than 70% missing data from both training and test datasets. Print the names of dropped columns and verify the new dataset dimensions to ensure data quality standards are maintained.

In [ ]:
# TODO Remove columns with excessive missing data

threshold = 0.7
drop_cols = train.columns[train.isnull().sum() / len(train) > threshold]

print(f"Dropping columns with >{threshold*100}% missing data:")
print(list(drop_cols))

train.drop(columns=drop_cols, inplace=True)
test.drop(columns=drop_cols, inplace=True)

print(f"New training data shape: {train.shape}")

## 3.2 Forward-backward fill for time-series missing data

Time-series data has temporal dependencies where adjacent time points are correlated. For air quality measurements, if PM2.5 is missing at 3 PM, the values at 2 PM and 4 PM are likely good estimates. Forward fill uses the last known value (2 PM → 3 PM), while backward fill uses the next known value (4 PM → 3 PM). Combining both methods ensures gaps at the beginning and end of time series are also filled, maximizing data retention while preserving temporal patterns.

- **Forward fill (ffill)**: Use the last known value
- **Backward fill (bfill)**: Use the next known value

**IMPORTANT**: We must apply this **within each city** because each city has its own air quality patterns, measurement conditions, and temporal dynamics, so imputation should respect these geographic boundaries.

**Documentation references:**
- [pandas.DataFrame.ffill()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html)
- [pandas.DataFrame.bfill()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.bfill.html)
- [pandas.DataFrame.groupby().transform()](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.GroupBy.transform.html)

**Exercise:** Build temporal imputation function for geographic groups
 
Each city has unique pollution patterns that shouldn't be mixed. Creating a function ensures consistent data processing across the entire pipeline.
 
Build a function that sorts data by group and date, then applies forward fill followed by backward fill within each group using pandas `.ffill()` and `.bfill()` methods. 
This ensures missing values are filled using temporally adjacent data points while respecting geographic boundaries (each city's data is processed separately).

In [ ]:
# TODO Define a function to fill missing values using forward and backward fill
 
def forward_back_fill(df, cols, group_col, date_col):
    """
    Fill missing values using forward and backward fill within each group.

    Parameters:
    - df: DataFrame to process
    - cols: List of columns to fill
    - group_col: Column to group by (e.g., 'city')
    - date_col: Date column for sorting

    Returns:
    - DataFrame with missing values filled
    """
    # Sort by group and date to ensure proper temporal order
    df = df.sort_values(by=[group_col, date_col])

    # Apply forward fill then backward fill for each col within each group
    for col in cols:
        df[col] = df.groupby(group_col)[col].transform(lambda x: x.ffill().bfill())

    return df

**Exercise:** Process missing data within each city independently for both training and test datasets.

Loop through each city and for each dataset extract that city's data, identify columns with missing values, apply the forward-backward fill function, then update the original dataset. This ensures that each city's missing values are filled using only that city's own temporal patterns, maintaining geographic consistency across the entire pipeline.

In [ ]:
# TODO Process both datasets city by city

for city in train['city'].unique():
    # Process training data for this city
    train_city = train[train['city'] == city].copy()
    train_cols_missing = train_city.columns[train_city.isnull().any()]
    if len(train_cols_missing) > 0:
        train_city = forward_back_fill(train_city, train_cols_missing, 
                                     group_col="city", date_col="date")
        train.update(train_city)
    
    # Process test data for this city
    test_city = test[test['city'] == city].copy()
    test_cols_missing = test_city.columns[test_city.isnull().any()]
    if len(test_cols_missing) > 0:
        test_city = forward_back_fill(test_city, test_cols_missing, 
                                    group_col="city", date_col="date")
        test.update(test_city)

In [ ]:
# Verify that missing values have been handled

print("Missing values after cleaning:")
print(f"Train: {train.isnull().sum().sum()}")
print(f"Test: {test.isnull().sum().sum()}")

***
# 4. Understanding Cross-Validation Strategy for Grouped Data
 
Our data has an important characteristic: observations are grouped by city. Cities might have different climate patterns, pollution sources, or measurement conditions. Using random train/test splits could put data from the same city in both training and validation sets, leading to data leakage and overly optimistic performance estimates. We need a specialized cross-validation approach that respects these geographical groupings.

## 4.1 The Problem with Standard Cross-Validation

Standard k-fold cross-validation randomly splits data into folds, which can create data leakage when observations are naturally grouped. For geographical data like ours, this means a model could see data from Lagos in training and then be tested on different Lagos data points, artificially inflating performance metrics because cities have consistent internal patterns.

**Documentation references:**
- [Cross-validation overview](https://scikit-learn.org/stable/modules/cross_validation.html)
- [Data leakage in machine learning](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators-for-grouped-data)

**Exercise:** Understand why city grouping matters for validation

Each city has unique pollution characteristics that create internal consistency. Random splitting would allow models to exploit these city-specific patterns rather than learning generalizable relationships.

Examine the unique cities in your training data and consider how many data points each city contributes. Think about what would happen if training data included some Lagos measurements and validation data included other Lagos measurements - would this give us realistic performance estimates for predicting air quality in completely new cities?

In [ ]:
# TODO Print the number of measurements by city in our dataset

city_distribution = train.groupby('city').size().reset_index(name='measurements')
city_distribution

## 4.2 GroupKFold Solution

GroupKFold ensures that entire cities are either in training OR validation, never both. This provides more realistic performance estimates by testing the model's ability to generalize to completely unseen locations, which better reflects real-world deployment scenarios.

**Documentation references:**
- [GroupKFold documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html)
- [Grouped data cross-validation](https://scikit-learn.org/stable/modules/cross_validation.html#group-k-fold)

**Exercise:** Create city-based cross-validation folds

GroupKFold prevents data leakage by keeping all data from each city together. This tests our model's ability to generalize to new cities rather than memorizing city-specific patterns.
Use GroupKFold with 4 splits to create folds where each fold contains complete cities. Set the groups parameter to the city column so that all measurements from the same city stay together. Store the fold assignments in a new 'folds' column for later use.

In [ ]:
# TODO Create city-based folds using GroupKFold
# Each fold will contain complete cities, not random samples

gkf = GroupKFold(n_splits=4)
train['folds'] = np.nan

# The 'groups' parameter tells GroupKFold which samples belong to the same group (city)
for fold, (train_idx, val_idx) in enumerate(gkf.split(train, groups=train['city']), 1):
    train.loc[val_idx, 'folds'] = fold

# Convert to integer type for easier handling
train['folds'] = train['folds'].astype(int)


**Exercise:** Verify fold distribution and balance

Understanding how cities are distributed across folds helps ensure our validation strategy is sound. Ideally, each fold should have reasonable representation and similar target distributions.
Examine which cities are assigned to each fold and calculate basic statistics for the target variable (PM2.5) across folds. Check that folds are reasonably balanced in terms of sample size and target variable distribution to ensure robust cross-validation.

In [ ]:
# TODO Check which cities are in each fold and how many samples per fold

fold_distribution = train.groupby(['folds', 'city']).size().reset_index(name='count')

print("Distribution of cities across folds:")
fold_distribution

In [ ]:
# Let's also check the target distribution across folds
# We want to ensure folds are reasonably balanced

fold_stats = train.groupby('folds')[Config.target_col].agg(['count', 'mean', 'std']).round(2)

print(Config.target_col + " statistics by fold:")
print(fold_stats)

***
# 5. Feature Engineering for Time-Series and Geographic Data
 
Feature engineering is crucial for time-series data. We need to extract meaningful patterns from dates and create geographic features.

## 5.1 Temporal Feature Engineering
 
Air quality data exhibits strong temporal patterns: pollution levels vary by season (winter heating, summer ozone), day of week (weekday traffic vs weekend), and time of day (rush hours). Extracting these cyclical patterns as features helps machine learning models capture these relationships explicitly rather than trying to infer them from raw timestamps.

**Documentation references:**
- [Time series / date functionality](https://pandas.pydata.org/docs/user_guide/timeseries.html)

**Exercise:** Understanding temporal patterns in air quality
 
Before extracting features, we need to understand why temporal patterns matter for air quality prediction. 
Different times of year, days of week, and hours show different pollution patterns due to:
- Seasonal effects: winter heating, summer photochemical reactions
- Weekly cycles: higher traffic on weekdays vs weekends  
- Daily patterns: rush hour peaks, industrial activity schedules
 
Create the `extract_temporal_features` function that extracts these meaningful time components from datetime data to help our model understand when pollution is typically higher or lower.

In [ ]:
# TODO Define a function to extract temporal features from a datetime column

def extract_temporal_features(df, date_col='date'):
    """
    Extract temporal features from a datetime column to capture cyclical patterns.
    
    Air quality varies with seasonal changes, weekly patterns (weekdays vs weekends),
    and daily cycles. These temporal features help models understand when pollution
    levels are typically higher or lower.

    Parameters:
    - df: DataFrame containing the date column
    - date_col: Name of the datetime column to extract features from

    Returns:
    - DataFrame with additional temporal features (year, month, day, quarter, week and dayofweek)
    """
    # Ensure the date column is datetime type for proper extraction
    df[date_col] = pd.to_datetime(df[date_col])

    # Extract basic temporal components that affect air quality
    df['year'] = df[date_col].dt.year           # Long-term trends
    df['month'] = df[date_col].dt.month         # Seasonal patterns (heating season, etc.)
    df['day'] = df[date_col].dt.day             # Monthly patterns
    df['quarter'] = df[date_col].dt.quarter     # Quarterly business cycles
    df['week'] = df[date_col].dt.isocalendar().week.astype("int64")  # Weekly patterns
    df['dayofweek'] = df[date_col].dt.dayofweek # 0=Monday, 6=Sunday (weekday vs weekend)

    return df

**Exercise:** Apply temporal feature engineering to both datasets

Use your function to extract temporal features from both training and test datasets. This ensures both datasets have the same feature structure for consistent model training and prediction.

In [ ]:
# TODO Apply temporal feature to train & test sets

train = extract_temporal_features(train)
test = extract_temporal_features(test)

In [ ]:
# Display the new temporal features

print("New temporal features created:")
temporal_cols = ['year', 'month', 'day', 'quarter', 'week', 'dayofweek']
print(train[['date'] + temporal_cols].head())

## 5.2 Geographic Feature Engineering
 
Air quality monitoring stations at different geographic locations can have distinct pollution profiles due to local factors like traffic density, industrial proximity, topography, and meteorological conditions. Creating location-specific identifiers helps models learn these location-based patterns.

**Documentation references:**
- [pandas.DataFrame.astype()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html)
- [String concatenation in pandas](https://pandas.pydata.org/docs/user_guide/text.html#concatenation)

**Exercise 1:** Understanding the importance of geographic features
 
Different monitoring stations can have vastly different pollution levels even within the same city due to local factors like proximity to highways, industrial zones, or natural barriers.
By creating unique location identifiers, we help the model distinguish between these micro-environments and learn location-specific pollution patterns.

Implement the `extract_geographic_features` function that combines latitude and longitude coordinates to create unique `location` identifier (`latitude_longitude` format) for each monitoring station.

In [ ]:
# TODO Define a function to extract geographic features from latitude and longitude coordinates
 
def extract_geographic_features(df):
    """
    Create geographic features from latitude and longitude coordinates.
    
    Different monitoring locations have unique pollution characteristics based on
    their surroundings (urban vs rural, near highways, industrial areas, etc.).
    Combining coordinates creates unique location identifiers that help models
    learn location-specific pollution patterns.

    Parameters:
    - df: DataFrame with site_latitude and site_longitude columns

    Returns:
    - DataFrame with additional geographic feature (location identifier)
    """
    # Create a unique location identifier by combining coordinates
    # This allows the model to learn location-specific patterns
    df['location'] = df['site_latitude'].astype('str') + '_' + df['site_longitude'].astype('str')

    return df


**Exercise 2:** Apply geographic feature engineering
 
Use your function to create location identifiers for both training and test datasets.
This will help the model understand that different coordinate pairs represent different pollution environments with potentially different baseline levels and patterns.

In [ ]:
# TODO Apply geographic feature to train & test sets

train = extract_geographic_features(train)
test = extract_geographic_features(test)

In [ ]:
print(f"Created {train['location'].nunique()} unique locations")

## 5.3 Categorical Feature Encoding
 
Machine learning algorithms require numerical inputs, but our dataset contains categorical variables like location identifiers and dates. We need to convert these text-based categories into numerical representations while ensuring consistency between training and test datasets. **Label Encoding** assigns a unique integer to each category. 

**Documentation references:**
- [Label Encoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

**Exercise 1:** Understanding the need for categorical encoding
 
Machine learning models like Linear Regression work with numbers, not text. When we have categorical variables like location names or dates, we need to convert them to numerical format. However, we must ensure that the same category gets the same number in both training and test datasets to maintain consistency.

Implement the `encode_categorical_features` function that uses Label Encoding to convert categorical variables to integers while maintaining consistency between datasets.

In [ ]:
# TODO Write a function to encode categorical features
def encode_categorical_features(train_df, test_df):
    """
    Encode categorical features for machine learning models.
    
    Most ML algorithms require numerical inputs, but our data contains categorical
    variables (location names, dates) that need conversion to numbers. We use 
    Label Encoding to assign unique integers to each category while ensuring
    consistent encoding between training and test sets.

    Parameters:
    - train_df, test_df: Training and test DataFrames with categorical features

    Returns:
    - train_encoded: Encoded training DataFrame
    - test_encoded: Encoded test DataFrame  
    - feature_cols: List of column names to use as model features
    """
    # Combine datasets to ensure consistent encoding across train/test
    # This prevents issues where test set has categories not seen in training
    combined_data = pd.concat([train_df, test_df], ignore_index=True)

    # Initialize label encoder for consistent categorical-to-numerical conversion
    le = LabelEncoder()

    # Identify categorical columns that need encoding
    # These are high-cardinality categories (many unique values)
    categorical_cols = ['location', 'date']

    # Apply label encoding: convert each unique category to a unique integer
    for col in categorical_cols:
        if col in combined_data.columns:
            combined_data[col] = le.fit_transform(combined_data[col])

    # Split back into separate train and test datasets
    train_encoded = combined_data[combined_data['id'].isin(train_df['id'])]
    test_encoded = combined_data[combined_data['id'].isin(test_df['id'])]

    # Define feature columns (exclude target variable and metadata)
    exclude_cols = [Config.target_col, Config.id_col, 'folds', 'country', 'city', 
                   'site_id', 'site_latitude', 'site_longitude']
    feature_cols = [col for col in train_encoded.columns if col not in exclude_cols]

    return train_encoded, test_encoded, feature_cols



**Exercise 2:** Apply categorical encoding to prepare data for modeling

Use your function to encode categorical features in both datasets and identify which columns should be used as features for model training (excluding target variables and metadata that shouldn't influence predictions).

In [ ]:
# TODO Apply the function to encode categorical features train and test sets

train_processed, test_processed, feature_columns = encode_categorical_features(train, test)

In [ ]:
print(f"Total feature columns for modeling: {len(feature_columns)}")
print(f"Feature columns: {feature_columns}...")

# Verify encoding worked correctly
print(f"\nData shapes after encoding:")
print(f"Training data: {train_processed.shape}")
print(f"Test data: {test_processed.shape}")

***
# 6. Baseline Model Development
 
Building a baseline model is essential before exploring complex algorithms. A simple Linear Regression model provides a reference point for measuring improvement and helps validate that our data preprocessing and cross-validation strategy work correctly. Starting with interpretable models also helps us understand which features are most important for air quality prediction.

## 6.1 Train-Validation Split Using Fold Strategy

We'll use our city-based folds to create a proper train-validation split that respects geographical boundaries. This ensures our baseline performance estimates are realistic and comparable to our final cross-validation results.

**Exercise:** Create geographically-separated train and validation sets

Using fold-based splitting ensures our model evaluation reflects real-world deployment conditions. We need distinct city groups for training and validation to avoid data leakage.


```
📊 FOLD-BASED DATA SPLITTING STRATEGY

Original Dataset (train_processed)
┌─────────────────────────────────────────────────────────────┐
│  FOLD 1    │  FOLD 2    │  FOLD 3    │  FOLD 4    │
│  City A    │  City B    │  City C    │  City D    │
│  [data]    │  [data]    │  [data]    │  [data]    │
└─────────────────────────────────────────────────────────────┘

                    ↓ SPLIT ↓

TRAINING DATA (Folds 1, 3, 4)          VALIDATION DATA (Fold 2)
┌─────────────────────────────────┐    ┌─────────────────────┐
│  City A + City C + City D       │    │      City B         │
│                                 │    │                     │
│  ↓ SEPARATE FEATURES & TARGET   │    │  ↓ SEPARATE FEATURES│
│                                 │    │    & TARGET         │
│  X_train: [features matrix]     │    │  X_val: [features]  │
│  y_train: [PM2.5 values]        │    │  y_val: [PM2.5]     │
└─────────────────────────────────┘    └─────────────────────┘

🎯 RESULT: Complete geographical separation
   → No city appears in both training and validation
   → Prevents data leakage and ensures realistic evaluation
```


Use folds 1, 3, 4 for training and fold 2 for validation to create train_data and val_data subsets. Then prepare feature matrices `(X_train, X_val)` and target vectors `(y_train, y_val)` using the feature_columns list from our preprocessing pipeline.

In [ ]:
# TODO Select the relevant folds for train and val data
training_folds = [1, 3, 4]
validation_fold = [2]

train_data = train_processed[train_processed['folds'].isin(training_folds)]
val_data = train_processed[train_processed['folds'].isin(validation_fold)]

In [ ]:
print(f"Training data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")

In [ ]:
# TODO Prepare feature matrices X_train, y_train and target vectors X_val, y_val

X_train = train_data[feature_columns]
y_train = train_data[Config.target_col]
X_val = val_data[feature_columns]
y_val = val_data[Config.target_col]

In [ ]:
print(f"Training features shape: {X_train.shape}")
print(f"Training target shape: {y_train.shape}")

## 6.2 Linear Regression Model Training and Evaluation

Linear Regression assumes linear relationships between features and target variables. While this assumption may not hold perfectly for air quality data, it provides an interpretable baseline and helps identify the most predictive features through coefficient analysis.

Documentation references:
- [Linear Regression in scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- [Mean Squared Error](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error)
- [R² score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination)

**Exercise:** Train baseline Linear Regression model

Establishing baseline performance helps gauge whether more complex models provide meaningful improvements. Simple models also help identify obvious data quality issues early in the pipeline.

Train a LinearRegression model on the training data, make predictions on the validation set, and calculate three key performance metrics: Mean Squared Error (MSE), Root Mean Squared Error (RMSE), and R² score to establish our baseline performance benchmark.


In [ ]:
# TODO Train the model

baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

In [ ]:
# TODO Make predictions

y_pred = baseline_model.predict(X_val)

**Exercise:** Analyze baseline model performance metrics

Understanding baseline performance provides context for evaluating more sophisticated approaches. These metrics help determine if our preprocessing pipeline is working correctly and set expectations for model improvement.

Display the calculated performance metrics (MSE, RMSE, R²) with clear labels and appropriate decimal precision. Interpret what these numbers mean in the context of air quality prediction - RMSE shows average prediction error in µg/m³, while R² indicates the proportion of variance explained by the model.

In [ ]:
# TODO Calculate performance metrics

mse = mean_squared_error(y_val, y_pred)
rmse = root_mean_squared_error(y_val, y_pred)
r2_score = baseline_model.score(X_val, y_val)

In [ ]:
print("Baseline Model Performance:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2_score:.4f}")

---

# 7. Feature Selection for Model Improvement
 
Not all features are equally important for predicting air quality. Feature selection can improve model performance by focusing on the most relevant variables, reduce overfitting by eliminating noise, and enhance interpretability by identifying the key factors that drive PM2.5 levels. We'll explore two complementary approaches: statistical selection and recursive elimination.


## 7.1 SelectKBest Feature Selection

SelectKBest uses statistical tests to identify features with the strongest linear relationships to the target variable. For regression problems, it employs F-statistics to measure how much each feature explains the variance in PM2.5 levels. This method is fast and effective for identifying obviously important features, making it ideal for initial feature screening.

**Documentation references:**
- [SelectKBest documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)
- [f_regression scoring function](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html)
- [Feature selection guide](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection)

**Exercise:** Apply statistical feature selection with `SelectKBest`

Statistical tests quickly identify features with strong predictive power. This approach helps eliminate obviously irrelevant variables before applying more sophisticated selection methods.

Initialize `SelectKBest` with `f_regression` scoring and `k=15` features. Fit the selector on training data, transform both training and validation sets, then create a summary DataFrame showing feature names, their F-scores, and selection status to understand which variables are most predictive.

In [ ]:
# Select top 15 features

k_best_features = 15  

In [ ]:
# TODO Initialize and fit the selector

selector_kbest = SelectKBest(score_func=f_regression, k=k_best_features)
X_train_kbest = selector_kbest.fit_transform(X_train, y_train)
X_val_kbest = selector_kbest.transform(X_val)

In [ ]:
# TODO Get selected feature names and their scores

selected_features_kbest = np.array(feature_columns)[selector_kbest.get_support()]
feature_scores = selector_kbest.scores_

In [ ]:
# TODO Create a summary DataFrame

feature_importance_df = pd.DataFrame({
    'Feature': feature_columns,
    'Score': feature_scores,
    'Selected': selector_kbest.get_support()
}).sort_values('Score', ascending=False)

In [ ]:
print("Top 10 features by SelectKBest score:")
print(feature_importance_df.head(10))

**Exercise:** Evaluate `SelectKBest` model performance against baseline

Comparing feature selection results to the baseline helps determine if reducing dimensionality improves generalization. Sometimes fewer, more relevant features outperform using all available variables.

Train a new LinearRegression model using only the `SelectKBest` features, make predictions on the validation set, and calculate `MSE`, `RMSE` and `R²` metrics. Compare these results to the baseline performance to assess whether feature selection improved model quality.

In [ ]:
# TODO Train a new model using only the selected features
 
kbest_model = LinearRegression()
kbest_model.fit(X_train_kbest, y_train)

y_pred_kbest = kbest_model.predict(X_val_kbest)


In [ ]:
# TODO Compute performance metrics for the SelectKBest model

mse_kbest = mean_squared_error(y_val, y_pred_kbest)
rmse_kbest = root_mean_squared_error(y_val, y_pred_kbest)
r2_kbest = kbest_model.score(X_val_kbest, y_val)

In [ ]:
print("SelectKBest Model Performance:")
print(f"MSE: {mse_kbest:.2f} (baseline: {mse:.2f})")
print(f"RMSE: {rmse_kbest:.2f} (baseline: {rmse:.2f})")
print(f"R² Score: {r2_kbest:.4f} (baseline: {r2_score:.4f})")

## 7.2 Recursive Feature Elimination (RFE)

RFE takes a different approach by iteratively training models and eliminating the least important features based on model coefficients. This method considers feature interactions and dependencies that univariate tests might miss, potentially finding more nuanced feature combinations that work well together for air quality prediction.

**Documentation references:**
- [Recursive feature elimination guide](https://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination)
- [RFE documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)

**Exercise:** Apply model-based feature selection with RFE

`RFE` considers how features work together within the model context. Unlike statistical tests, it accounts for feature interactions and multicollinearity when making selection decisions.

Initialize `RFE` with `LinearRegression` as the estimator and select `15` features. Fit `RFE` on training data, transform both datasets, then examine the selected features and their model coefficients to understand which variables the model considers most important in combination.

In [ ]:
# Select top 15 features
rfe_features = 15  

In [ ]:
# TODO Initialize RFE with LinearRegression as the estimator
estimator = LinearRegression()
selector_rfe = RFE(estimator=estimator, n_features_to_select=rfe_features)

In [ ]:
# TODO Fit RFE and transform features
X_train_rfe = selector_rfe.fit_transform(X_train, y_train)
X_val_rfe = selector_rfe.transform(X_val)

In [ ]:
# TODO Get selected features and their rankings
selected_features_rfe = np.array(feature_columns)[selector_rfe.support_]
feature_rankings = selector_rfe.ranking_

In [ ]:
print("Features selected by RFE:")
rfe_df = pd.DataFrame({
    'Feature': selected_features_rfe,
    'Coefficient': selector_rfe.estimator_.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(rfe_df)

**Exercise:** Compare RFE performance with previous approaches

Model-based selection might identify different feature combinations than statistical tests. Comparing both approaches helps determine which selection strategy works better for our specific air quality prediction task.

Train a `LinearRegression` model using RFE-selected features, calculate performance metrics (`MSE`, `RMSE`, `R²`), and compare results with both baseline and SelectKBest approaches to identify the most effective feature selection method for this dataset.

In [ ]:
# TODO Train a new model using only the selected features

rfe_model = LinearRegression()
rfe_model.fit(X_train_rfe, y_train)

y_pred_rfe = rfe_model.predict(X_val_rfe)


In [ ]:
# TODO Compute performance metrics for the RFE model

mse_rfe = mean_squared_error(y_val, y_pred_rfe)
rmse_rfe = root_mean_squared_error(y_val, y_pred_rfe)
r2_rfe = rfe_model.score(X_val_rfe, y_val)

# Display RFE model performance
print("RFE Model Performance:")
print(f"RMSE: {rmse_rfe:.2f}")
print(f"R² Score: {r2_rfe:.4f}")

***
# 8. Cross-Validation Evaluation
 
Single train-validation splits can give misleading performance estimates, especially with geographical data where one city might be easier or harder to predict than others. Cross-validation provides more robust and reliable performance estimates by testing our model across all city combinations. This comprehensive evaluation helps us understand model consistency and identify potential overfitting to specific geographical patterns.

## 8.1 Implementing K-Fold Cross-Validation with Best Features

We'll evaluate our best-performing feature selection method across all four folds to get a comprehensive view of model performance. This approach ensures our performance estimates aren't biased by the particular choice of validation cities and provides confidence intervals for our metrics.

Documentation references:
- [Cross-validation in scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html)
- [Model evaluation best practices](https://scikit-learn.org/stable/modules/model_evaluation.html#cross-validation)
- [Performance metric interpretation](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)

**Exercise:** Select optimal feature set from previous experiments

Choosing the best-performing feature selection method ensures our cross-validation uses the most effective variable combination. This decision impacts all subsequent model evaluations and comparisons.

Compare the `RMSE` results from `SelectKBest` and `RFE` approaches from the previous section. Choose the method that achieved lower validation RMSE and assign its selected features to a variable called `selected_features` for use in cross-validation.

In [ ]:
# Choose the best performing feature selection method
selected_features = selected_features_rfe

**Exercise:** Implement comprehensive cross-validation across all folds

Testing across all fold combinations provides robust performance estimates. Each city serves as validation data exactly once, ensuring fair evaluation across different geographical contexts.

Create a loop that iterates through folds 1-4, using each fold as validation while training on the remaining three folds. For each iteration, prepare the feature matrices using `selected_features`, train a `LinearRegression` model, make predictions, and calculate `RMSE` and `R²` scores to build a comprehensive performance profile.


In [ ]:
# TODO Implement comprehensive cross-validation across all folds

# Initialize lists to store results
fold_rmse_scores = []
fold_r2_scores = []

print("\nCross-validation results:")

for fold in [1, 2, 3, 4]:
    # Create train/validation split for current fold
    train_fold = train_processed[train_processed['folds'] != fold]
    val_fold = train_processed[train_processed['folds'] == fold]
    
    # Prepare data
    X_train_cv = train_fold[selected_features]
    y_train_cv = train_fold[Config.target_col]
    X_val_cv = val_fold[selected_features]
    y_val_cv = val_fold[Config.target_col]
    
    # Train model
    cv_model = LinearRegression()
    cv_model.fit(X_train_cv, y_train_cv)
    
    # Make predictions and evaluate
    y_pred_cv = cv_model.predict(X_val_cv)
    
    fold_rmse = root_mean_squared_error(y_val_cv, y_pred_cv)
    fold_r2 = cv_model.score(X_val_cv, y_val_cv)
    
    fold_rmse_scores.append(fold_rmse)
    fold_r2_scores.append(fold_r2)
    
    print(f"Fold {fold}: RMSE = {fold_rmse:.2f}, R² = {fold_r2:.4f}")

## 8.2 Statistical Analysis of Cross-Validation Results

Analyzing the distribution of performance across folds helps us understand model stability and reliability. High variance between folds might indicate overfitting or geographical bias, while consistent performance suggests robust generalization capability.

**Exercise:** Calculate aggregate performance statistics with confidence measures

Understanding performance variability across folds helps assess model reliability. Standard deviations indicate how consistent the model performs across different geographical contexts.

Calculate the `mean` and `standard deviation` of `RMSE` and `R²` scores across all folds. Display the results in the format `mean ± std` to show both central tendency and variability, providing a complete picture of model performance consistency.


In [ ]:
# TODO Calculate and print overall performance

mean_rmse = np.mean(fold_rmse_scores)
std_rmse = np.std(fold_rmse_scores)
mean_r2 = np.mean(fold_r2_scores)
std_r2 = np.std(fold_r2_scores)

print("\n")
print(f"Average RMSE: {mean_rmse:.2f} ± {std_rmse:.2f}")
print(f"Average R²: {mean_r2:.4f} ± {std_r2:.4f}")

***
# 9. Results Analysis and Interpretation

Understanding model performance goes beyond simple metrics. We need to visualize prediction quality, interpret feature importance, and contextualize our results within real-world air quality standards. This analysis helps determine if our model is ready for practical deployment and identifies areas for potential improvement.

## 9.1 Model Performance Visualization

Visual analysis reveals patterns that summary statistics might miss. Scatter plots of actual vs predicted values show prediction accuracy across different pollution levels, while residual plots help identify systematic errors or heteroscedasticity that could indicate model limitations.

Documentation references:
- [Residuals vs. Fits Plot](https://online.stat.psu.edu/stat462/node/117/)
- [Residual Plot: Definition and Examples](https://www.statisticshowto.com/residual-plot/)

**Exercise:** Create diagnostic plots for prediction quality assessment

Visual diagnostics reveal model strengths and weaknesses across different prediction ranges. Good models show tight clustering around the diagonal line and random residual patterns.

Create two diagnostic subplots to evaluate prediction quality:
- **Actual vs Predicted scatter plot**: Points close to the diagonal line (y=x) indicate accurate predictions, while points far from the line show prediction errors
- **Residuals plot**: Calculate residuals as (actual - predicted) values, then plot them against predicted values - random scatter around zero indicates good model fit, while patterns suggest systematic errors

In [ ]:
# TODO Create a comparison plot of actual vs predicted values

# Use the last fold's predictions for visualization
plt.figure(figsize=(12, 5))

# Scatter plot of actual vs predicted
plt.subplot(1, 2, 1)
plt.scatter(y_val_cv, y_pred_cv, alpha=0.6, s=20)
plt.plot([y_val_cv.min(), y_val_cv.max()], [y_val_cv.min(), y_val_cv.max()], 'r--', lw=2)
plt.xlabel('Actual PM2.5')
plt.ylabel('Predicted PM2.5')
plt.title('Actual vs Predicted PM2.5')


# Residuals plot
plt.subplot(1, 2, 2)
residuals = y_val_cv - y_pred_cv
plt.scatter(y_pred_cv, residuals, alpha=0.6, s=20)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted PM2.5')
plt.ylabel('Residuals')
plt.title('Residuals Plot')


plt.tight_layout()
plt.show()

## 9.2 Feature Importance Analysis

Understanding which features drive predictions helps validate model logic and provides insights for air quality management. Important features should align with domain knowledge about pollution sources and meteorological influences on air quality.

Documentation references:
- [Feature importance interpretation](https://scikit-learn.org/stable/modules/feature_selection.html#feature-importance-scores)
- [Linear regression coefficients](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares)


**Exercise:** Visualize most influential features for model predictions

Feature importance analysis validates model logic and provides actionable insights. Important features should make scientific sense for air quality prediction.

Create a horizontal bar chart showing the top 10 most important features. Use the method that performed better. Sort features by importance and add appropriate labels to help interpret which variables most strongly influence PM2.5 predictions.

In [ ]:
# TODO Create a visualization of the most important features

if rmse_kbest < rmse_rfe:
    # Use SelectKBest scores
    importance_data = feature_importance_df[feature_importance_df['Selected']].head(10)
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(importance_data)), importance_data['Score'])
    plt.yticks(range(len(importance_data)), importance_data['Feature'])
    plt.xlabel('Feature Importance Score')
    plt.title('Top 10 Features (SelectKBest)')
else:
    # Use RFE coefficients
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(rfe_df)), abs(rfe_df['Coefficient']))
    plt.yticks(range(len(rfe_df)), rfe_df['Feature'])
    plt.xlabel('Absolute Coefficient Value')
    plt.title('Feature Importance (RFE)')

plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 9.3 Contextual Performance Interpretation

Raw performance metrics need context to be meaningful. Comparing our prediction errors to WHO air quality guidelines and typical PM2.5 ranges helps assess whether our model accuracy is sufficient for practical applications like public health warnings or policy decisions.

Documentation references:
- [WHO Air Quality Guidelines](https://iris.who.int/bitstream/handle/10665/345329/9789240034228-eng.pdf)
- [PM2.5 health impact thresholds](https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf) (page 4)

**Exercise:** Interpret model performance within air quality context

Understanding prediction accuracy relative to health thresholds helps determine model utility. Prediction errors should be small compared to differences between air quality categories.

Create a comprehensive summary showing final RMSE and R² with their interpretations. Include WHO air quality guidelines for PM2.5 levels and explain how our prediction accuracy relates to these public health thresholds.


In [ ]:

# TODO Create summary statistics

print("Model Performance Summary:")
print("=" * 40)
print(f"Final RMSE: {mean_rmse:.2f} µg/m³")
print(f"Final R² Score: {mean_r2:.4f}")
print()
print("Interpretation:")
print(f"- On average, our predictions are off by {mean_rmse:.1f} µg/m³")
print(f"- The model explains {mean_r2*100:.1f}% of the variance in PM2.5 levels")
print(f"- Standard deviation of RMSE across folds: {std_rmse:.2f}")

In [ ]:
# TODO Find in documents some context information for PM2.5 levels (WHO guidelines)

print()
print("WHO Air Quality Guidelines (PM2.5):")
print("- Good: 0-12 µg/m³")
print("- Moderate: 12-35 µg/m³") 
print("- Unhealthy: 35-55 µg/m³")
print("- Very Unhealthy: 55+ µg/m³")

# Make a conclusion based on the model performance and WHO guidelines

With prediction errors of 27.1 µg/m³, our model cannot reliably distinguish between WHO air quality categories, as this error spans multiple health risk levels (e.g., 'Good' to 'Unhealthy' range).